# 01c: Calculus Refresh

**Week 1, Day 5** | Foundations

**Prerequisites**: 01a (tensors, autograd), 01b (vectors, matrices)

---

## Learning Objectives

By the end of this notebook, you will be able to:

- [ ] Compute derivatives symbolically and numerically
- [ ] Understand gradients as "direction of steepest ascent"
- [ ] Apply the chain rule (the heart of backpropagation)
- [ ] Implement gradient descent from scratch
- [ ] **MILESTONE**: Optimize the Rosenbrock function

---

In [ ]:
# Imports
import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

# Config
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

np.random.seed(42)

---

# Part 1: Derivatives

## Layer 1: Intuition — Rate of change

A derivative answers: **"If I nudge the input a little, how much does the output change?"**

**Physical analogies**:

| Function | Derivative | Meaning |
|----------|------------|---------|
| Position | Velocity | How fast am I moving? |
| Velocity | Acceleration | How fast is my speed changing? |
| Cost | Marginal cost | How much more does one extra unit cost? |
| Loss | Gradient | Which way should I adjust parameters? |

**Visual intuition**: The derivative is the **slope of the tangent line**. A steep slope means small input changes cause big output changes.

```
      y │      ╱
        │     ╱ ← tangent line (derivative)
        │   .╱.
        │  ╱.   .
        │ ╱  . ← curve f(x)
        │╱    .
      ──┼─────────── x
        │
```

**Why derivatives for ML?**: The loss function measures "how wrong we are." Its derivative tells us which direction to adjust weights to be less wrong.

## Layer 2: Code + Visualization

In [ ]:
# Numerical derivative: the finite difference approximation

def numerical_derivative(f, x, h=1e-7):
    """
    Compute derivative of f at x using central difference.
    
    The central difference formula:
    f'(x) ≈ (f(x+h) - f(x-h)) / (2h)
    
    This is more accurate than the forward difference (f(x+h) - f(x)) / h.
    """
    return (f(x + h) - f(x - h)) / (2 * h)

# Test on f(x) = x²
f = lambda x: x**2
f_prime_exact = lambda x: 2*x  # We know this analytically

test_points = [0, 1, 2, 3]
print("f(x) = x²,  f'(x) = 2x\n")
for x in test_points:
    numerical = numerical_derivative(f, x)
    exact = f_prime_exact(x)
    print(f"x = {x}: numerical f'({x}) = {numerical:.6f}, exact = {exact}")

In [ ]:
# Visualize derivative as tangent line slope

def plot_function_with_tangent(f, f_prime, x0, x_range=(-3, 3), title=""):
    """
    Plot function and its tangent line at x0.
    """
    x = np.linspace(x_range[0], x_range[1], 200)
    y = f(x)
    
    # Tangent line: y = f(x0) + f'(x0) * (x - x0)
    slope = f_prime(x0)
    tangent = f(x0) + slope * (x - x0)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(x, y, 'b-', linewidth=2, label='f(x)')
    ax.plot(x, tangent, 'r--', linewidth=2, label=f'Tangent at x={x0} (slope={slope:.2f})')
    ax.scatter([x0], [f(x0)], color='red', s=100, zorder=5)
    
    ax.set_xlim(x_range)
    ax.set_ylim(-2, 10)
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)
    ax.legend()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(title or f"f(x) with tangent at x = {x0}")
    ax.grid(True, alpha=0.3)
    
    return ax

# f(x) = x²
f = lambda x: x**2
f_prime = lambda x: 2*x

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, x0 in enumerate([-1, 0, 2]):
    ax = axes[i]
    x = np.linspace(-3, 3, 200)
    ax.plot(x, f(x), 'b-', linewidth=2, label='f(x) = x²')
    slope = f_prime(x0)
    tangent = f(x0) + slope * (x - x0)
    ax.plot(x, tangent, 'r--', linewidth=2, label=f"Tangent (slope={slope})")
    ax.scatter([x0], [f(x0)], color='red', s=100, zorder=5)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-2, 10)
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)
    ax.legend(fontsize=9)
    ax.set_title(f"x = {x0}, f'({x0}) = {slope}")
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Common derivatives you should know

functions = [
    ("x²", lambda x: x**2, lambda x: 2*x, "2x"),
    ("x³", lambda x: x**3, lambda x: 3*x**2, "3x²"),
    ("sin(x)", np.sin, np.cos, "cos(x)"),
    ("cos(x)", np.cos, lambda x: -np.sin(x), "-sin(x)"),
    ("eˣ", np.exp, np.exp, "eˣ"),
    ("ln(x)", np.log, lambda x: 1/x, "1/x"),
]

print("Common derivatives:\n")
print(f"{'f(x)':<12} {'f\'(x)':<12} {'Numerical at x=2':<20} {'Exact at x=2':<15}")
print("-" * 60)
for name, f, f_prime, f_prime_name in functions:
    x = 2.0
    numerical = numerical_derivative(f, x)
    exact = f_prime(x)
    print(f"{name:<12} {f_prime_name:<12} {numerical:<20.6f} {exact:<15.6f}")

## Layer 3: CS Speak

### Derivative Computation Methods

| Method | Formula | Error | Use case |
|--------|---------|-------|----------|
| **Symbolic** | Apply rules (power rule, chain rule) | Exact | Small expressions |
| **Numerical** | $(f(x+h) - f(x-h))/(2h)$ | $O(h^2)$ | Black-box functions |
| **Automatic (autograd)** | Track computation graph | Exact (to floating point) | Deep learning |

### Numerical Stability

Choosing $h$ in numerical differentiation is tricky:
- Too large: truncation error (approximation is poor)
- Too small: round-off error (floating point precision)

Sweet spot: $h \approx \sqrt{\epsilon_{machine}} \approx 10^{-8}$ for float64.

### Autograd Advantage

Numerical differentiation scales as $O(n)$ for $n$ parameters (need $2n$ function evaluations).

Autograd (reverse-mode) scales as $O(1)$ in the number of outputs — one backward pass gives ALL gradients. This is why we can train networks with millions of parameters.

## Layer 4: Mathematical Formalism

### Definition

The derivative of $f: \mathbb{R} \to \mathbb{R}$ at $x$ is:

$$f'(x) = \lim_{h \to 0} \frac{f(x+h) - f(x)}{h}$$

if this limit exists.

### Derivative Rules

**Linearity**: $(af + bg)' = af' + bg'$

**Product rule**: $(fg)' = f'g + fg'$

**Quotient rule**: $(f/g)' = (f'g - fg')/g^2$

**Chain rule**: $(f \circ g)'(x) = f'(g(x)) \cdot g'(x)$

### Power Rule

$$\frac{d}{dx}x^n = nx^{n-1}$$

for any real $n$.

---

# Part 2: Gradients

## Layer 1: Intuition — Which way is uphill?

A derivative tells you the slope for a function of ONE variable. But what if your function depends on MULTIPLE variables?

The **gradient** is the collection of all partial derivatives — it tells you the direction of steepest increase.

**Physical analogy**: You're on a hilly terrain in fog. You can't see far, but you can feel the ground under your feet. The gradient tells you: "The steepest uphill direction is THAT way."

- **Gradient** points uphill (direction of steepest ascent)
- **Negative gradient** points downhill (direction of steepest descent)
- **Magnitude of gradient** tells you how steep the slope is

To minimize a loss function, we walk in the direction of the **negative gradient**.

## Layer 2: Code + Visualization

In [ ]:
# Partial derivatives: derivative w.r.t. one variable, holding others fixed

# f(x, y) = x² + y²
def f(x, y):
    return x**2 + y**2

# Partial derivatives
def df_dx(x, y):
    return 2*x  # ∂f/∂x: treat y as constant, differentiate w.r.t. x

def df_dy(x, y):
    return 2*y  # ∂f/∂y: treat x as constant, differentiate w.r.t. y

# The gradient is [∂f/∂x, ∂f/∂y]
def gradient(x, y):
    return np.array([df_dx(x, y), df_dy(x, y)])

# Test
point = (3, 4)
print(f"f(x, y) = x² + y²")
print(f"At point {point}:")
print(f"  f{point} = {f(*point)}")
print(f"  ∂f/∂x = {df_dx(*point)}")
print(f"  ∂f/∂y = {df_dy(*point)}")
print(f"  ∇f = {gradient(*point)}")

In [ ]:
# Visualize the gradient field on a surface

def f(x, y):
    return x**2 + y**2

def gradient(x, y):
    return np.array([2*x, 2*y])

# Create meshgrid
x = np.linspace(-3, 3, 50)
y = np.linspace(-3, 3, 50)
X, Y = np.meshgrid(x, y)
Z = f(X, Y)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: 3D surface
ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.plot_surface(X, Y, Z, cmap=cm.viridis, alpha=0.8, edgecolor='none')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f(x, y)')
ax.set_title('f(x, y) = x² + y²\n(a "bowl" with minimum at origin)')

# Right: Contour plot with gradient arrows
ax = axes[1]
contour = ax.contourf(X, Y, Z, levels=20, cmap=cm.viridis, alpha=0.8)
plt.colorbar(contour, ax=ax, label='f(x,y)')

# Gradient arrows at selected points
arrow_points = [(-2, -2), (-2, 2), (2, -2), (2, 2), (0, 2), (2, 0), (1, 1)]
for px, py in arrow_points:
    grad = gradient(px, py)
    # Scale arrows for visibility
    scale = 0.15
    ax.quiver(px, py, grad[0]*scale, grad[1]*scale, 
              color='red', width=0.01, scale=1, scale_units='xy')

ax.scatter(*zip(*arrow_points), c='red', s=30, zorder=5)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Gradient field (red arrows point uphill)\n'
             'To minimize: follow negative gradient (downhill)')
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

print("\nNotice: Gradients point away from the minimum (origin).")
print("To minimize, we walk OPPOSITE to the gradient.")

In [ ]:
# More interesting function: f(x, y) = sin(x) + cos(y)

def f(x, y):
    return np.sin(x) + np.cos(y)

def gradient(x, y):
    return np.array([np.cos(x), -np.sin(y)])

# Create meshgrid
x = np.linspace(-np.pi, np.pi, 50)
y = np.linspace(-np.pi, np.pi, 50)
X, Y = np.meshgrid(x, y)
Z = f(X, Y)

fig, ax = plt.subplots(figsize=(10, 8))

# Contour plot
contour = ax.contourf(X, Y, Z, levels=20, cmap=cm.coolwarm, alpha=0.8)
plt.colorbar(contour, ax=ax, label='f(x,y)')

# Gradient arrows on a coarser grid
arrow_x = np.linspace(-np.pi, np.pi, 8)
arrow_y = np.linspace(-np.pi, np.pi, 8)
AX, AY = np.meshgrid(arrow_x, arrow_y)

# Compute gradients
GX = np.cos(AX)
GY = -np.sin(AY)

# Normalize for visibility
magnitude = np.sqrt(GX**2 + GY**2)
GX_norm = GX / (magnitude + 1e-10)
GY_norm = GY / (magnitude + 1e-10)

ax.quiver(AX, AY, GX_norm, GY_norm, color='black', alpha=0.7, scale=20)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('f(x, y) = sin(x) + cos(y) with gradient field\n'
             'Arrows point toward increasing values')
ax.set_aspect('equal')

plt.show()

## Layer 3: CS Speak

### Gradient Computation

```python
# NumPy: manual
def gradient(x, y):
    return np.array([df_dx(x, y), df_dy(x, y)])

# PyTorch: automatic
x = torch.tensor([1.0, 2.0], requires_grad=True)
y = (x**2).sum()  # scalar output
y.backward()
print(x.grad)  # Gradient w.r.t. x
```

### Properties of Gradients

- **Direction**: Points toward steepest ascent
- **Magnitude**: $\|\nabla f\|$ = rate of steepest ascent
- **Perpendicular to level sets**: Gradients are orthogonal to contour lines

### Directional Derivative

The derivative in direction $\mathbf{u}$ (unit vector):

$$D_{\mathbf{u}} f = \nabla f \cdot \mathbf{u}$$

Maximum when $\mathbf{u}$ aligns with $\nabla f$. This is why gradient points uphill!

## Layer 4: Mathematical Formalism

### Definition

For $f: \mathbb{R}^n \to \mathbb{R}$, the **gradient** at $\mathbf{x}$ is:

$$\nabla f(\mathbf{x}) = \begin{bmatrix} \frac{\partial f}{\partial x_1} \\ \frac{\partial f}{\partial x_2} \\ \vdots \\ \frac{\partial f}{\partial x_n} \end{bmatrix}$$

### Partial Derivative

$$\frac{\partial f}{\partial x_i} = \lim_{h \to 0} \frac{f(x_1, \ldots, x_i + h, \ldots, x_n) - f(x_1, \ldots, x_i, \ldots, x_n)}{h}$$

### Gradient Properties

**Linearity**: $\nabla(af + bg) = a\nabla f + b\nabla g$

**Product rule**: $\nabla(fg) = f\nabla g + g\nabla f$

**Chain rule**: For $h = f \circ g$, $\nabla h = (\nabla f)(g) \cdot J_g$

where $J_g$ is the Jacobian matrix of $g$.

### Directional Derivative Theorem

For unit vector $\mathbf{u}$:

$$D_\mathbf{u} f(\mathbf{x}) = \nabla f(\mathbf{x}) \cdot \mathbf{u} \leq \|\nabla f(\mathbf{x})\|$$

with equality when $\mathbf{u} = \nabla f / \|\nabla f\|$.

**Proof**: By Cauchy-Schwarz. ∎

---

# Part 3: The Chain Rule

## Layer 1: Intuition — Rates multiply through compositions

If function A doubles its input, and function B triples its input, then A(B(x)) multiplies by 6.

More generally: if you change $x$ a little, how much does $f(g(x))$ change?

1. First, $x$ changes $g(x)$ by rate $g'(x)$
2. Then, that change in $g$ changes $f$ by rate $f'(g(x))$
3. Total rate = multiply them: $f'(g(x)) \cdot g'(x)$

**Physical analogy**: Gears in a machine.
- Gear A turns 2× for each turn of the input
- Gear B turns 3× for each turn of A
- Total: 6× from input to B

**Why this is the heart of deep learning**: A neural network is a composition of many functions:

$$\text{output} = f_n(f_{n-1}(\cdots f_1(\text{input})))$$

The chain rule lets us compute how each parameter affects the output by "chaining" through all the intermediate layers. This is **backpropagation**.

## Layer 2: Code + Visualization

In [ ]:
# Chain rule example: f(g(x)) where f(u) = u², g(x) = sin(x)

# Composition: h(x) = sin²(x)
def h(x):
    return np.sin(x)**2

# Chain rule: h'(x) = f'(g(x)) · g'(x) = 2·sin(x) · cos(x) = sin(2x)
def h_prime(x):
    return 2 * np.sin(x) * np.cos(x)  # = sin(2x)

# Verify numerically
x_test = np.pi / 4
numerical = numerical_derivative(h, x_test)
exact = h_prime(x_test)

print(f"h(x) = sin²(x)")
print(f"h'(x) = 2·sin(x)·cos(x) = sin(2x)")
print(f"\nAt x = π/4:")
print(f"  Numerical h'(x) = {numerical:.6f}")
print(f"  Exact h'(x)     = {exact:.6f}")
print(f"  sin(2·π/4) = sin(π/2) = {np.sin(np.pi/2):.6f}")

In [ ]:
# Visualize the chain rule as a computation graph

# Simple example: y = (x + 2)²
# Let's compute dy/dx step by step

x = 3.0  # Input

# Forward pass (compute the function)
a = x + 2       # a = x + 2 = 5
y = a**2        # y = a² = 25

print("Forward pass:")
print(f"  x = {x}")
print(f"  a = x + 2 = {a}")
print(f"  y = a² = {y}")

# Backward pass (compute gradients via chain rule)
dy_dy = 1.0        # Gradient of y w.r.t. itself
dy_da = 2 * a      # d(a²)/da = 2a
da_dx = 1.0        # d(x+2)/dx = 1

# Chain rule: dy/dx = dy/da · da/dx
dy_dx = dy_da * da_dx

print("\nBackward pass (chain rule):")
print(f"  dy/dy = {dy_dy}")
print(f"  dy/da = 2a = 2·{a} = {dy_da}")
print(f"  da/dx = 1")
print(f"  dy/dx = dy/da · da/dx = {dy_da} · {da_dx} = {dy_dx}")

# Verify: d/dx[(x+2)²] = 2(x+2) = 2·5 = 10
print(f"\nVerification: d/dx[(x+2)²] = 2(x+2) = {2*(x+2)}")

In [ ]:
# A more complex example: simple neural network forward/backward pass

# Network: y = σ(w·x + b) where σ is sigmoid
# We want dy/dw and dy/db

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    s = sigmoid(z)
    return s * (1 - s)

# Parameters
x = 2.0    # Input
w = 0.5    # Weight
b = -1.0   # Bias

# Forward pass
z = w * x + b        # Linear combination
y = sigmoid(z)       # Activation

print("Forward pass:")
print(f"  x = {x}, w = {w}, b = {b}")
print(f"  z = w·x + b = {w}·{x} + {b} = {z}")
print(f"  y = σ(z) = {y:.4f}")

# Backward pass
dy_dz = sigmoid_derivative(z)  # σ'(z) = σ(z)(1-σ(z))
dz_dw = x                       # ∂(wx+b)/∂w = x
dz_db = 1                       # ∂(wx+b)/∂b = 1

# Chain rule
dy_dw = dy_dz * dz_dw
dy_db = dy_dz * dz_db

print("\nBackward pass (chain rule):")
print(f"  dy/dz = σ'(z) = σ(z)(1-σ(z)) = {dy_dz:.4f}")
print(f"  dz/dw = x = {dz_dw}")
print(f"  dz/db = 1")
print(f"  dy/dw = dy/dz · dz/dw = {dy_dz:.4f} · {dz_dw} = {dy_dw:.4f}")
print(f"  dy/db = dy/dz · dz/db = {dy_dz:.4f} · {dz_db} = {dy_db:.4f}")

# Verify with PyTorch
print("\nVerification with PyTorch autograd:")
w_t = torch.tensor(w, requires_grad=True)
b_t = torch.tensor(b, requires_grad=True)
x_t = torch.tensor(x)

z_t = w_t * x_t + b_t
y_t = torch.sigmoid(z_t)
y_t.backward()

print(f"  dy/dw (PyTorch) = {w_t.grad.item():.4f}")
print(f"  dy/db (PyTorch) = {b_t.grad.item():.4f}")

## Layer 3: CS Speak

### Backpropagation = Chain Rule + Dynamic Programming

Backpropagation is efficient because it:
1. Computes derivatives in reverse order (output → input)
2. Reuses intermediate results (dynamic programming)

**Complexity**:
- Forward pass: $O(\text{model size})$
- Backward pass: $O(\text{model size})$ — same order!

Compare to numerical differentiation: $O(\text{parameters} \times \text{model size})$

### Computation Graph

```
Forward:  x → [×w] → [+b] → [σ] → y

Backward: dy/dx ← dy/dz·dz/dx ← dy/da·da/dz ← dy/dy
```

Each node stores its local gradient. Backprop multiplies them together.

### Common Local Gradients

| Operation | Local gradient |
|-----------|---------------|
| $y = x + c$ | $\partial y/\partial x = 1$ |
| $y = cx$ | $\partial y/\partial x = c$ |
| $y = x^2$ | $\partial y/\partial x = 2x$ |
| $y = e^x$ | $\partial y/\partial x = e^x$ |
| $y = \sigma(x)$ | $\partial y/\partial x = \sigma(x)(1-\sigma(x))$ |
| $y = \text{ReLU}(x)$ | $\partial y/\partial x = \mathbf{1}_{x > 0}$ |

## Layer 4: Mathematical Formalism

### Univariate Chain Rule

For $h = f \circ g$:

$$\frac{dh}{dx} = \frac{df}{dg} \cdot \frac{dg}{dx} = f'(g(x)) \cdot g'(x)$$

### Multivariate Chain Rule

For $f: \mathbb{R}^m \to \mathbb{R}$ and $\mathbf{g}: \mathbb{R}^n \to \mathbb{R}^m$:

$$\frac{\partial (f \circ \mathbf{g})}{\partial x_i} = \sum_{j=1}^{m} \frac{\partial f}{\partial g_j} \cdot \frac{\partial g_j}{\partial x_i}$$

In matrix form:

$$\nabla_\mathbf{x}(f \circ \mathbf{g}) = \mathbf{J}_\mathbf{g}^T \nabla_\mathbf{g} f$$

where $\mathbf{J}_\mathbf{g}$ is the Jacobian matrix $(\partial g_j / \partial x_i)$.

### Vector-Jacobian Product (VJP)

The key operation in backpropagation:

$$\mathbf{v}^T \mathbf{J}$$

where $\mathbf{v}$ is the upstream gradient and $\mathbf{J}$ is the local Jacobian.

This can be computed without explicitly forming $\mathbf{J}$ — crucial for efficiency!

---

# Part 4: Gradient Descent

## Layer 1: Intuition — Following the downhill path

Gradient descent is remarkably simple:

1. **Look around**: Compute the gradient (which way is uphill?)
2. **Walk downhill**: Take a step in the opposite direction
3. **Repeat**: Keep walking until you reach a valley

**Physical analogy**: A ball rolling down a bowl. It naturally finds the bottom.

The **learning rate** (step size) is crucial:
- Too small: Takes forever to converge
- Too large: Overshoots and bounces around (or diverges!)
- Just right: Smooth convergence

**Why "descent" and not just random search?** The gradient tells us the BEST direction to move. Any other direction makes less progress per step.

## Layer 2: Code + Visualization

In [ ]:
# Gradient descent from scratch

def gradient_descent(f, grad_f, x0, lr=0.1, n_steps=100, tol=1e-8):
    """
    Minimize f starting from x0 using gradient descent.
    
    Args:
        f: Function to minimize
        grad_f: Gradient of f
        x0: Starting point (numpy array)
        lr: Learning rate (step size)
        n_steps: Maximum iterations
        tol: Convergence tolerance
    
    Returns:
        x_opt: Optimized point
        history: List of (x, f(x)) at each step
    """
    x = x0.copy()
    history = [(x.copy(), f(x))]
    
    for i in range(n_steps):
        grad = grad_f(x)
        
        # Check convergence
        if np.linalg.norm(grad) < tol:
            print(f"Converged at step {i} (gradient norm < {tol})")
            break
        
        # Update: x = x - lr * gradient
        x = x - lr * grad
        history.append((x.copy(), f(x)))
    
    return x, history

In [ ]:
# Example: Minimize f(x, y) = x² + y² (simple bowl)

def f(xy):
    return xy[0]**2 + xy[1]**2

def grad_f(xy):
    return np.array([2*xy[0], 2*xy[1]])

# Run gradient descent
x0 = np.array([4.0, 3.0])
x_opt, history = gradient_descent(f, grad_f, x0, lr=0.2, n_steps=50)

print(f"Starting point: {x0}, f = {f(x0)}")
print(f"Optimal point: {x_opt}, f = {f(x_opt):.6f}")
print(f"True minimum: [0, 0], f = 0")

In [ ]:
# Visualize the optimization path

# Create meshgrid for contour plot
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)
Z = X**2 + Y**2

fig, ax = plt.subplots(figsize=(10, 8))

# Contour plot
contour = ax.contour(X, Y, Z, levels=15, cmap='viridis')
ax.clabel(contour, inline=True, fontsize=8)

# Optimization path
path = np.array([h[0] for h in history])
ax.plot(path[:, 0], path[:, 1], 'ro-', markersize=4, linewidth=1, label='Gradient descent path')
ax.scatter(path[0, 0], path[0, 1], c='green', s=100, zorder=5, label='Start')
ax.scatter(path[-1, 0], path[-1, 1], c='red', s=100, zorder=5, label='End')
ax.scatter(0, 0, c='blue', s=100, marker='*', zorder=5, label='True minimum')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title(f'Gradient Descent on f(x,y) = x² + y²\nLearning rate = 0.2, {len(history)} steps')
ax.legend()
ax.set_aspect('equal')

plt.show()

In [ ]:
# Learning rate matters! Comparison

learning_rates = [0.01, 0.1, 0.5, 0.9]
x0 = np.array([4.0, 3.0])

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for ax, lr in zip(axes.flat, learning_rates):
    # Run gradient descent
    x_opt, history = gradient_descent(f, grad_f, x0, lr=lr, n_steps=50)
    
    # Contour plot
    contour = ax.contour(X, Y, Z, levels=15, cmap='viridis', alpha=0.5)
    
    # Path
    path = np.array([h[0] for h in history])
    ax.plot(path[:, 0], path[:, 1], 'ro-', markersize=3, linewidth=1)
    ax.scatter(path[0, 0], path[0, 1], c='green', s=80, zorder=5)
    ax.scatter(0, 0, c='blue', s=80, marker='*', zorder=5)
    
    # Loss curve
    losses = [h[1] for h in history]
    
    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)
    ax.set_aspect('equal')
    ax.set_title(f'lr = {lr}\nFinal f = {losses[-1]:.4f} ({len(history)} steps)')

plt.suptitle('Effect of Learning Rate on Gradient Descent', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Loss curves comparison

fig, ax = plt.subplots(figsize=(10, 6))

for lr in [0.01, 0.1, 0.3, 0.5]:
    x_opt, history = gradient_descent(f, grad_f, x0, lr=lr, n_steps=100)
    losses = [h[1] for h in history]
    ax.semilogy(losses, label=f'lr = {lr}')

ax.set_xlabel('Iteration')
ax.set_ylabel('Loss (log scale)')
ax.set_title('Convergence Speed vs Learning Rate')
ax.legend()
ax.grid(True, alpha=0.3)

plt.show()

## Layer 3: CS Speak

### Gradient Descent Algorithm

```python
def gradient_descent(params, lr, n_steps):
    for _ in range(n_steps):
        loss = compute_loss(params)
        grad = compute_gradient(params)
        params = params - lr * grad
    return params
```

### Variants

| Variant | Update rule | Key idea |
|---------|-------------|----------|
| **Vanilla GD** | $\theta \leftarrow \theta - \eta \nabla L$ | Basic version |
| **SGD** | Same, but on mini-batch | Stochastic, faster per step |
| **Momentum** | $v \leftarrow \beta v + \nabla L$; $\theta \leftarrow \theta - \eta v$ | Accumulate velocity |
| **Adam** | Adaptive learning rates | Most popular in practice |

### Hyperparameters

- **Learning rate ($\eta$)**: Most important! Start with 0.001-0.01
- **Batch size**: Larger = more stable gradients, but slower per epoch
- **Momentum ($\beta$)**: Usually 0.9

### Convergence Conditions

For convex functions with L-Lipschitz gradients, GD with $\eta < 1/L$ converges.

## Layer 4: Mathematical Formalism

### Gradient Descent Update

$$\mathbf{x}_{t+1} = \mathbf{x}_t - \eta \nabla f(\mathbf{x}_t)$$

where $\eta > 0$ is the learning rate.

### Why It Works (Descent Lemma)

If $f$ has L-Lipschitz continuous gradient:

$$f(\mathbf{y}) \leq f(\mathbf{x}) + \nabla f(\mathbf{x})^T(\mathbf{y} - \mathbf{x}) + \frac{L}{2}\|\mathbf{y} - \mathbf{x}\|^2$$

Setting $\mathbf{y} = \mathbf{x} - \eta \nabla f(\mathbf{x})$:

$$f(\mathbf{x}_{t+1}) \leq f(\mathbf{x}_t) - \eta(1 - \frac{L\eta}{2})\|\nabla f(\mathbf{x}_t)\|^2$$

For $\eta < 2/L$, we have guaranteed descent: $f(\mathbf{x}_{t+1}) < f(\mathbf{x}_t)$.

### Convergence Rate

For $\mu$-strongly convex, $L$-smooth functions with $\eta = 1/L$:

$$f(\mathbf{x}_t) - f(\mathbf{x}^*) \leq \left(1 - \frac{\mu}{L}\right)^t (f(\mathbf{x}_0) - f(\mathbf{x}^*))$$

This is **linear convergence** — error decreases by a constant factor each iteration.

---

# Week 1 Milestone: Rosenbrock Function

The **Rosenbrock function** is a classic optimization test:

$$f(x, y) = (1 - x)^2 + 100(y - x^2)^2$$

It has:
- A global minimum at $(1, 1)$ where $f = 0$
- A curved, narrow valley that's easy to find but hard to follow

Your task: **Implement gradient descent and visualize the optimization path.**

In [ ]:
# Rosenbrock function and gradient

def rosenbrock(xy):
    """The Rosenbrock function."""
    x, y = xy
    return (1 - x)**2 + 100 * (y - x**2)**2

def rosenbrock_grad(xy):
    """Gradient of the Rosenbrock function."""
    x, y = xy
    dx = -2 * (1 - x) - 400 * x * (y - x**2)
    dy = 200 * (y - x**2)
    return np.array([dx, dy])

# Verify gradient with numerical differentiation
test_point = np.array([0.5, 0.5])
h = 1e-7
numerical_grad = np.array([
    (rosenbrock(test_point + np.array([h, 0])) - rosenbrock(test_point - np.array([h, 0]))) / (2*h),
    (rosenbrock(test_point + np.array([0, h])) - rosenbrock(test_point - np.array([0, h]))) / (2*h)
])
analytic_grad = rosenbrock_grad(test_point)

print(f"At {test_point}:")
print(f"  Numerical gradient: {numerical_grad}")
print(f"  Analytic gradient:  {analytic_grad}")
print(f"  Match: {np.allclose(numerical_grad, analytic_grad)}")

In [ ]:
# Run gradient descent on Rosenbrock

x0 = np.array([-1.0, 1.0])
lr = 0.001  # Small learning rate needed for this challenging function
n_steps = 10000

x_opt, history = gradient_descent(rosenbrock, rosenbrock_grad, x0, lr=lr, n_steps=n_steps)

print(f"Starting point: {x0}, f = {rosenbrock(x0):.4f}")
print(f"Final point: {x_opt}, f = {rosenbrock(x_opt):.6f}")
print(f"True minimum: [1, 1], f = 0")
print(f"Distance to optimum: {np.linalg.norm(x_opt - np.array([1, 1])):.6f}")

In [ ]:
# Visualize the optimization path on Rosenbrock

# Create meshgrid
x = np.linspace(-2, 2, 200)
y = np.linspace(-1, 3, 200)
X, Y = np.meshgrid(x, y)
Z = (1 - X)**2 + 100 * (Y - X**2)**2

fig, ax = plt.subplots(figsize=(12, 10))

# Contour plot (log scale for better visualization)
levels = np.logspace(-1, 3, 20)
contour = ax.contour(X, Y, Z, levels=levels, cmap='viridis')
ax.clabel(contour, inline=True, fontsize=8, fmt='%.0f')

# Optimization path
path = np.array([h[0] for h in history])
# Subsample for visibility
step = max(1, len(path) // 200)
path_sub = path[::step]

ax.plot(path_sub[:, 0], path_sub[:, 1], 'r.-', markersize=2, linewidth=0.5, alpha=0.7,
        label=f'GD path ({len(history)} steps)')
ax.scatter(path[0, 0], path[0, 1], c='green', s=150, zorder=5, label='Start', edgecolor='white')
ax.scatter(path[-1, 0], path[-1, 1], c='red', s=150, zorder=5, label='End', edgecolor='white')
ax.scatter(1, 1, c='blue', s=200, marker='*', zorder=5, label='True minimum (1,1)', edgecolor='white')

ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title(f'Gradient Descent on Rosenbrock Function\n'
             f'lr = {lr}, steps = {len(history)}, final f = {rosenbrock(x_opt):.4f}',
             fontsize=14)
ax.legend(loc='upper left')

plt.show()

In [ ]:
# Loss curve

losses = [h[1] for h in history]

fig, ax = plt.subplots(figsize=(10, 6))
ax.semilogy(losses)
ax.set_xlabel('Iteration')
ax.set_ylabel('Loss (log scale)')
ax.set_title('Rosenbrock Optimization: Loss vs Iteration')
ax.grid(True, alpha=0.3)
plt.show()

print(f"\nMILESTONE COMPLETE!")
print(f"You implemented gradient descent from scratch and optimized the Rosenbrock function.")
print(f"Final loss: {losses[-1]:.6f}")
print(f"Final position: {x_opt}")

---

# Exercises

## Exercise 1: Gradient Visualization

Plot $f(x, y) = \sin(x) + \cos(y)$ as a 3D surface. Then create a 2D contour plot with gradient arrows at several points.

In [ ]:
# Exercise 1: Gradient Visualization

def exercise1_gradient_viz():
    """
    Create a 3D surface plot and 2D contour plot with gradients
    for f(x, y) = sin(x) + cos(y).
    """
    # TODO: Define f(x, y)
    # TODO: Define gradient (analytical: [cos(x), -sin(y)])
    # TODO: Create meshgrid
    # TODO: Plot 3D surface
    # TODO: Plot 2D contours with gradient arrows
    pass

exercise1_gradient_viz()

## Exercise 2: Learning Rate Experiment

Run gradient descent on $f(x) = x^4 - 3x^3 + 2$ starting from $x_0 = 4$.

Try learning rates: 0.001, 0.01, 0.05, 0.1

Questions:
1. What happens when the learning rate is too large?
2. What's the optimal learning rate?
3. Where are the local minima of this function?

In [ ]:
# Exercise 2: Learning Rate Experiment

def exercise2_learning_rate():
    """
    Experiment with learning rates on f(x) = x⁴ - 3x³ + 2.
    """
    # TODO: Define f(x) and f'(x)
    # f'(x) = 4x³ - 9x²
    
    # TODO: Run gradient descent with different learning rates
    # TODO: Plot the results
    # TODO: Answer the questions
    pass

exercise2_learning_rate()

## Exercise 3: Manual Backprop

For a simple 2-layer network:
$$y = \sigma(w_2 \cdot \sigma(w_1 \cdot x + b_1) + b_2)$$

Compute $\partial y / \partial w_1$ by hand using the chain rule. Then verify with PyTorch autograd.

In [ ]:
# Exercise 3: Manual Backprop

def exercise3_manual_backprop():
    """
    Compute gradient of 2-layer network by hand and verify with PyTorch.
    """
    # Network: y = σ(w2 · σ(w1 · x + b1) + b2)
    
    # Parameters
    x = 1.0
    w1, b1 = 0.5, 0.1
    w2, b2 = -0.3, 0.2
    
    # TODO: Forward pass (compute y)
    # TODO: Backward pass (compute dy/dw1 using chain rule)
    # Chain: y → σ → (w2·h + b2) → h → σ → (w1·x + b1)
    
    # TODO: Verify with PyTorch
    pass

exercise3_manual_backprop()

---

# Why This Matters

Everything in deep learning is gradient descent:

| Concept | How it uses gradients |
|---------|----------------------|
| **Training** | Minimize loss via gradient descent |
| **Backpropagation** | Chain rule to compute gradients efficiently |
| **Adam, SGD** | Variants of gradient descent |
| **Learning rate schedulers** | Adapt $\eta$ during training |
| **Regularization** | Add gradient penalty terms |

The Rosenbrock function you optimized is a toy problem, but it shares key challenges with real neural networks:
- Narrow valleys (ill-conditioning)
- Sensitivity to learning rate
- Need for many iterations

In Week 5-6, we'll see how dynamical systems and variational calculus generalize these ideas.

---

# Resources

**Essential**:
- [3Blue1Brown: Essence of Calculus](https://www.youtube.com/playlist?list=PLZHQObOWTQDMsr9K-rj53DwVRMYO3t5Yr) — Visual intuition
- [3Blue1Brown: Neural Networks](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) — Backprop visualization

**Deep Dives**:
- [Matrix Calculus for Deep Learning](https://explained.ai/matrix-calculus/) — Detailed reference
- [An Overview of Gradient Descent Optimization Algorithms](https://ruder.io/optimizing-gradient-descent/) — SGD variants

**Wiki**:
- [Glossary](../../wiki/glossary.md) — gradient, chain rule, learning rate, backpropagation

---

# Reflection Questions

Before moving to Week 2:

1. **Why is autograd more efficient than numerical differentiation?**
   - Think about the number of function evaluations needed...

2. **What happens if all eigenvalues of the Hessian (second derivative matrix) are positive?**
   - Hint: This relates to convexity and local minima.

3. **Why is gradient descent called "greedy"? What are its limitations?**
   - Think about local vs global minima...

---

# Week 1 Complete!

You now have the mathematical foundations for the rest of this curriculum:

✅ **Tensor manipulation** (NumPy/PyTorch)  
✅ **Linear algebra** (transformations, eigenthings, SVD)  
✅ **Calculus** (gradients, chain rule, optimization)  

**Key insight**: All of machine learning is optimization. We define a loss, compute gradients, and descend. The magic is in choosing what to optimize.

---

→ [Week 2: Embeddings](../../syllabus/week-02-embeddings.md)